In [ ]:
#Force google colab to switch to high ram mode. Run it only once. 
a = []
while(1):
    a.append('1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mtcnn
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as plt
import os
import cv2
from mtcnn import MTCNN
from random import shuffle;
from tqdm import tqdm;
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates

In [ ]:
Data=pd.read_csv('/content/drive/MyDrive/BMI_Face_Data/Final.csv')
images=[]
BMI=[]
weight=[]
height=[]
Age=[]

Checking the indexes not present in the directory


In [ ]:
from os import path
for i in range(len(Data)):
  if(path.exists("/content/drive/MyDrive/BMI_Face_Data/Images/"+str(Data.iloc[i,6]))):
    pass
  else:
    print(i)


146
150
207


In [ ]:
from os import path
for i in range(len(Data)):
  if(path.exists("/content/drive/MyDrive/BMI_Face_Data/Images/"+str(Data.iloc[i,7]))):
    pass
  else:
    print(i)

17
18
27
152
155
198
206


Creating Images,Weight,Height,BMI array for the indexes present in Directory

In [ ]:
#for first Image
for i in range(len(Data)):
  if i in [146,150,207]:
    pass
  else:
    try:
      img=cv2.imread("/content/drive/MyDrive/BMI_Face_Data/Images/"+str(Data.iloc[i,6]))
      img=cv2.resize(img,(256,256))
      images.append(img)
      weight.append(Data.iloc[i,3])
      height.append(Data.iloc[i,4])
      BMI.append(Data.iloc[i,5])
      Age.append(Data.iloc[i,1])
      print(i)
    except:
      print(f'Not {i}')



0
1
2
Not 3
Not 4
Not 5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
147
148
149
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#for Second Image
for i in range(len(Data)):
  if i in [17,18,27,152,155,198,206]:
    pass
  else:
    try:
      img=cv2.imread("/content/drive/MyDrive/BMI_Face_Data/Images/"+str(Data.iloc[i,7]))
      img=cv2.resize(img,(256,256))
      images.append(img)
      weight.append(Data.iloc[i,3])
      height.append(Data.iloc[i,4])
      BMI.append(Data.iloc[i,5])
      Age.append(Data.iloc[i,1])
      print(i)
    except:
      print(f'Not {i}')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
19
20
21
22
23
24
25
26
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
153
154
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
199
200
201
202
203
204
205
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283

In [ ]:
print(len(images))
print(len(weight))
print(len(height))
print(len(BMI))
print(len(Age))

1227
1227
1227
1227
1227


In [ ]:
images_=np.array(images)
weight_=np.array(weight)
height_=np.array(height)
BMI_=np.array(BMI)
Age=np.array(Age)

In [ ]:
np.save("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Images.npy",images_)
np.save("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Weight.npy",weight_)
np.save("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Height.npy",height_)
np.save("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/BMI.npy",BMI_)
np.save('/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Age.npy',Age)

In [ ]:

Images=np.load("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Images.npy")
Weight=np.load("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Weight.npy")
Height=np.load("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Height.npy")
BMI=   np.load("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/BMI.npy")
Age=   np.load("/content/drive/MyDrive/BMI_Face_Data/Data_without_Preprocessing_2/Age.npy")

In [ ]:
Images.shape

(1227, 256, 256, 3)

In [ ]:
Detector=MTCNN()
def preprocessing(img):
  out_img=Detector.detect_faces(img)
  x,y,w,h=out_img[0]["box"]
  img=img[y-10:y+h+10,x-10:x+w+10]
  img=cv2.resize(img,(224,224))
  return img

In [ ]:
def preprocessing_2(img):
  mp_face = mp.solutions.face_detection.FaceDetection(model_selection=1,min_detection_confidence=0.5)
  dframe= img

  image_rows, image_cols, _ = dframe.shape
  image_input = cv2.cvtColor(dframe, cv2.COLOR_BGR2RGB)
  results = mp_face.process(image_input)
  detection=results.detections[0]
  location = detection.location_data

  relative_bounding_box = location.relative_bounding_box
  rect_start_point = _normalized_to_pixel_coordinates(relative_bounding_box.xmin, relative_bounding_box.ymin, image_cols,image_rows)
  rect_end_point = _normalized_to_pixel_coordinates(relative_bounding_box.xmin + relative_bounding_box.width,relative_bounding_box.ymin + relative_bounding_box.height, image_cols,image_rows)

  xleft,ytop=rect_start_point
  xright,ybot=rect_end_point

  crop_img = image_input[ytop-10: ybot+10, xleft-10: xright+10]
  crop_img=cv2.resize(crop_img,(224,224))
  return crop_img

In [ ]:
def preprocessing_3(img):
    face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/BMI_Face_Data/haarcascade_frontalface_default.xml')
    grayscale=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(grayscale, 1.1, 8)
    for (x,y,w,h) in faces:
        crop_img=img[y-10:y+h+10,x-10:x+w+10]
        crop_img=cv2.resize(crop_img,(224,224))
    return crop_img

In [ ]:
cropped_image=[]
Weight_=[]
Height_=[]
BMI_=[]
Age_=[]
for i in range(len(Images)):
  try:
    cropped_image.append(preprocessing(Images[i]))
    Weight_.append(Weight[i])
    Height_.append(Height[i])
    BMI_.append(BMI[i])
    Age_.append(Age[i])
  except:
    try:
      cropped_image.append(preprocessing_2(Images[i]))
      Weight_.append(Weight[i])
      Height_.append(Height[i])
      BMI_.append(BMI[i])
      Age_.append(Age[i])
    except:
      try:
        cropped_image.append(preprocessing_3(Images[i]))
        Weight_.append(Weight[i])
        Height_.append(Height[i])
        BMI_.append(BMI[i])
        Age_.append(Age[i])
      except:
        print(i)


10
69
70
72
76
83
86
87
88
90
91
99
102
106
116
118
119
125
126
127
237
271
304
507
567
629
650
683
686
687
688
697
698
700
702
704
707
708
710
713
716
720
730
732
733
734
741
910
916
919
934
1053
1166


In [ ]:
cropped_image=np.array(cropped_image)
Weight_=np.array(Weight_)
Height_=np.array(Height_)
BMI_=np.array(BMI_)
Age_=np.array(Age_)

In [ ]:
cropped_image.shape

(1174, 224, 224, 3)

In [ ]:
np.save('/content/drive/MyDrive/BMI_Face_Data/Data_with_preprocessing_2/processed_img',cropped_image)
np.save('/content/drive/MyDrive/BMI_Face_Data/Data_with_preprocessing_2/processed_weight',Weight_)
np.save('/content/drive/MyDrive/BMI_Face_Data/Data_with_preprocessing_2/processed_height',Height_)
np.save('/content/drive/MyDrive/BMI_Face_Data/Data_with_preprocessing_2/processed_bmi',BMI_)
np.save('/content/drive/MyDrive/BMI_Face_Data/Data_with_preprocessing_2/processed_age',Age_)
